# Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.max_columns', None)

# Datensätze

In [2]:
df_matches = pd.read_csv("Datensätze/atp_matches_till_2022.csv")
df_players = pd.read_csv("Datensätze/atp_players_till_2022.csv")
df_rankings = pd.read_csv("Datensätze/atp_rankings_till_2022.csv")

# Hilfsfunktion(en)

In [3]:
def fillna_0_astype_int64(dataframe:pd.DataFrame, columns:list=["Siege", "Niederlagen"]):

    for column in columns:
        dataframe[column] = dataframe[column].fillna(0).astype("int64")
    
    return dataframe

In [4]:
def siegesquote_funktion(dataframe:pd.DataFrame):

    dataframe["Siegesquote"] = dataframe["Siege"] / (dataframe["Siege"]+ dataframe["Niederlagen"])

    return dataframe

<br>

## Fragestellungen

1. Gibt es „Angstgegner“-Nationen, gegen die die deutschen Spieler besonders schlecht abschneiden? Lässt sich das einfach durch die generelle Spielstärke der Spieler dieser Nationen erklären, oder gibt es Nationen, gegen die gerade deutsche Spieler schlecht abschneiden? Gibt es analog „Lieblingsgegner“-Nationen?
2. Gibt es „Angstgegner“ einzelner deutscher Spieler, d.h. Paarungen, bei denen der deutsche Spieler schlechter abschneidet, als es aufgrund der allgemeinen Spielstärke beider Spieler zu erwarten ist? Gibt es analog „Lieblingsgegner“ einzelner deutscher Spieler?
3. Gibt es Turniere, bei denen die Deutschen besonders gut oder besonders schlecht abschneiden? Wenn ja, ist ein regionales Muster oder eine andere Ursache zu erkennen?
4. Wie entwickelt sich der sportliche Erfolg mit dem Alter? Gilt das für die Spielstatistiken gleichermaßen wie für die Platzierung auf der Rangliste?
5. Sinkt die Wahrscheinlichkeit zu siegen, wenn das vorhergehende Spiel eines Spielers besonders lange gedauert hat?
6. Gibt es einen Zusammenhang dazwischen, wie man zuletzt gegen einen bestimmten Gegner gespielt hat, und wie das nächste Spiel gegen diesen Gegner ausgehen wird?
7. Wie gut lässt sich das Ergebnis eines Matches vorhersagen? Welches sind die wichtigsten Features für eine solche Prognose?

<br>

## Fragestellung 1:
Gibt es „Angstgegner“-Nationen, gegen die die deutschen Spieler besonders schlecht abschneiden? Lässt sich das einfach durch die generelle Spielstärke der Spieler dieser Nationen erklären, oder gibt es Nationen, gegen die gerade deutsche Spieler schlecht abschneiden? Gibt es analog „Lieblingsgegner“-Nationen?

In [5]:
df_matches

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1968-2029,Dublin,Grass,32,A,19680708,270,112411,NaN,NaN,Douglas Smith,U,NaN,AUS,NaN,110196,NaN,NaN,Peter Ledbetter,U,NaN,UNK,24.0,6-1 7-5,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-2029,Dublin,Grass,32,A,19680708,271,126914,NaN,NaN,Louis Pretorius,R,NaN,RSA,NaN,209536,NaN,NaN,Maurice Pollock,U,NaN,IRL,NaN,6-1 6-1,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-2029,Dublin,Grass,32,A,19680708,272,209523,NaN,NaN,Cecil Pedlow,U,NaN,IRL,NaN,209535,NaN,NaN,John Mulvey,U,NaN,IRL,NaN,6-2 6-2,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-2029,Dublin,Grass,32,A,19680708,273,100084,NaN,NaN,Tom Okker,R,178.0,NED,24.3,209534,NaN,NaN,Unknown Fearmon,U,NaN,NaN,NaN,6-1 6-1,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-2029,Dublin,Grass,32,A,19680708,274,100132,NaN,NaN,Armistead Neely,R,NaN,USA,21.3,209533,NaN,NaN,Harry Sheridan,U,NaN,IRL,NaN,6-2 6-4,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188156,2022-M-DC-2022-WG2-PO-GRE-JAM-01,Davis Cup WG2 PO: GRE vs JAM,Clay,4,D,20220304,4,209362,NaN,NaN,Aristotelis Thanos,R,NaN,GRE,20.8,209080,NaN,NaN,Blaise Bicknell,R,188.0,USA,20.2,6-4 6-4,3,RR,115.0,0.0,4.0,70.0,48.0,35.0,12.0,10.0,5.0,5.0,1.0,3.0,96.0,68.0,42.0,12.0,10.0,11.0,13.0,1103.0,9.0,1130.0,8.0
188157,2022-M-DC-2022-WG2-PO-GRE-JAM-01,Davis Cup WG2 PO: GRE vs JAM,Clay,4,D,20220304,5,202065,NaN,NaN,Petros Tsitsipas,U,185.0,GRE,21.5,202120,NaN,NaN,Rowland Phillips,R,NaN,JAM,28.0,4-6 6-4 6-4,3,RR,161.0,7.0,2.0,94.0,55.0,37.0,23.0,15.0,9.0,12.0,1.0,1.0,94.0,56.0,40.0,20.0,15.0,4.0,8.0,808.0,23.0,1390.0,4.0
188158,2022-M-DC-2022-WG2-PO-HKG-BEN-01,Davis Cup WG2 PO: HKG vs BEN,Hard,4,D,20220304,1,138846,NaN,NaN,Hong Kit Jack Wong,R,NaN,HKG,23.4,105278,NaN,NaN,Alexis Klegou,U,NaN,BEN,33.0,1-6 6-4 4-1 RET,3,RR,250.0,4.0,8.0,56.0,33.0,20.0,13.0,11.0,3.0,7.0,0.0,3.0,72.0,54.0,29.0,8.0,11.0,6.0,10.0,1059.0,10.0,1881.0,1.0
188159,2022-M-DC-2022-WG2-PO-HKG-BEN-01,Davis Cup WG2 PO: HKG vs BEN,Hard,4,D,20220304,2,209409,NaN,NaN,Coleman Wong,R,NaN,HKG,17.7,200583,NaN,NaN,Delmas Ntcha,R,NaN,BEN,21.7,6-4 6-4,3,RR,98.0,2.0,2.0,68.0,38.0,24.0,18.0,10.0,6.0,8.0,1.0,5.0,58.0,39.0,24.0,7.0,10.0,5.0,9.0,NaN,NaN,NaN,NaN


<br>

## Sieges- und Niederlagentabelle der Deutschen

In [6]:
#Dataframe für die Anzhal der Matches, bei denen Deutschland gegen die jeweilige Nation verloren hat
df_GER_loser = df_matches[df_matches.loser_ioc == "GER"].groupby(["loser_ioc", "winner_ioc"]).size().reset_index(name="loses_count")

#Dataframe für die Anzhal der Matches, bei denen Deutschland gegen die jeweilige Nation gewonnen hat
df_GER_winner = df_matches[df_matches.winner_ioc == "GER"].groupby(["loser_ioc","winner_ioc"]).size().reset_index(name="wins_count")

In [7]:
df_GER_winner

,loser_ioc,winner_ioc,wins_count
0,ALG,GER,5
1,ARG,GER,358
2,ARM,GER,10
3,AUS,GER,532
4,AUT,GER,201
...,...,...,...
87,UZB,GER,11
88,VEN,GER,20
89,VIE,GER,1
90,YUG,GER,13


In [8]:
df_GER_loser

,loser_ioc,winner_ioc,loses_count
0,GER,ALG,1
1,GER,ARG,478
2,GER,ARM,6
3,GER,AUS,543
4,GER,AUT,186
...,...,...,...
77,GER,USA,1464
78,GER,UZB,31
79,GER,VEN,3
80,GER,YUG,8


In [9]:
#Tabellen joinen
df_merge = pd.merge(df_GER_winner, df_GER_loser, how="outer", left_on="loser_ioc", right_on="winner_ioc")  

#Die Spalten zu einer kombinieren, die wegen des joins doppelt vorkommen 
df_merge['Deutschland'] = df_merge['winner_ioc_x'].combine_first(df_merge['loser_ioc_y'])
df_merge["Gegnernation"] = df_merge["loser_ioc_x"].combine_first(df_merge["winner_ioc_y"])

#Spalten umbenennen
df_merge.rename(columns={"wins_count": "Siege", "loses_count": "Niederlagen"}, inplace=True)

#Spalten entfernen, die nicht relevant sind
df_merge.drop(["winner_ioc_x", "winner_ioc_y",'loser_ioc_x', 'loser_ioc_y'], axis=1, inplace=True)

#NaN-Werte der Spalte "Niederlagen" und "Siege" durch 0 ersetzen und Datentyp zu "int" ändern
fillna_0_astype_int64(df_merge)

df_merge

,Siege,Niederlagen,Deutschland,Gegnernation
0,5,1,GER,ALG
1,358,478,GER,ARG
2,10,6,GER,ARM
3,532,543,GER,AUS
4,201,186,GER,AUT
...,...,...,...,...
88,20,3,GER,VEN
89,1,0,GER,VIE
90,13,8,GER,YUG
91,18,30,GER,ZIM


<br>

## Gibt es Angstgegner Nationen?

In [10]:
#Sigesquote von Deutschland als Spalte hinzufügen
siegesquote_funktion(df_merge)

#Zeilen rausfiltern, bei denen Deutschland insgesamt mehr als 10 Mal gegen die jeweilige Gegnernation gespielt hat.
#Damit können z.B. Sigesquoten von 100% ausgeschlossen werden, bei denen nur sehr wenige Matches satttgefunden haben
df_filtered = df_merge[(df_merge.Siege + df_merge.Niederlagen)>10]

#Zeilen, bei denen die Siegesquote von Deutschland < 50% ist
df_filtered = df_filtered[df_filtered["Siegesquote"] < 0.5].sort_values(by = "Siegesquote", ascending=True)
df_filtered


,Siege,Niederlagen,Deutschland,Gegnernation,Siegesquote
20,12,36,GER,CYP,0.250000
87,11,31,GER,UZB,0.261905
49,7,15,GER,LTU,0.318182
51,25,43,GER,MAR,0.367647
91,18,30,GER,ZIM,0.375000
61,17,26,GER,PAR,0.395349
13,14,21,GER,BUL,0.400000
74,169,245,GER,SUI,0.408213
15,87,122,GER,CHI,0.416268
72,95,133,GER,SRB,0.416667


In [11]:
#Die 5 geführchtetsten Gegnernationen
df_filtered.iloc[0:5]["Gegnernation"]

20    CYP
87    UZB
49    LTU
51    MAR
91    ZIM
Name: Gegnernation, dtype: object

<br>

### Visualisierung der Angstgegnernationen

In [12]:
fig = px.bar(df_filtered, x='Gegnernation', y='Siegesquote',
             title="Gegner, gegen die Deutschland eine Siegesquote von unter 50% hat",
             color='Siegesquote',
             color_continuous_scale='plasma',  
             labels={'Siegesquote': 'Siegesquote Deutschland'})
fig.show()

<br>

## Liegen die teilweise geringen Siegesquoten der Deutschan an der generellen Spielstärke der Gegnernation?

<br>

### Allgemeine Sigesquote aller Nationen

In [13]:
#Dataframe der Gewinnernationen
df_loser = df_matches.groupby(["loser_ioc", "winner_ioc"]).size().reset_index(name ="loses_count")

#Dataframe der Verlierernationen
df_winner = df_matches.groupby(["winner_ioc", "loser_ioc"]).size().reset_index(name="wins_count")

In [14]:
df_loser

,loser_ioc,winner_ioc,loses_count
0,AHO,BAH,1
1,AHO,BOL,2
2,AHO,BRA,4
3,AHO,CRC,2
4,AHO,DOM,2
...,...,...,...
5410,ZIM,UNK,1
5411,ZIM,URU,2
5412,ZIM,USA,113
5413,ZIM,ZAM,1


In [15]:
df_winner

,winner_ioc,loser_ioc,wins_count
0,AHO,BAH,2
1,AHO,BOL,2
2,AHO,CRC,2
3,AHO,DOM,2
4,AHO,ECU,1
...,...,...,...
5410,ZIM,UNK,3
5411,ZIM,USA,72
5412,ZIM,VEN,2
5413,ZIM,ZAM,3


In [16]:
#Summe der Siege pro Nation
df_winner = df_winner.groupby("winner_ioc")["wins_count"].sum().reset_index()
df_winner


,winner_ioc,wins_count
0,AHO,29
1,ALG,50
2,AND,3
3,ARG,8185
4,ARM,158
...,...,...
123,VEN,335
124,VIE,27
125,YUG,163
126,ZAM,7


In [17]:
#Summe der Niederlagen pro Nation 
df_loser = df_loser.groupby("loser_ioc")["loses_count"].sum().reset_index()
df_loser

,loser_ioc,loses_count
0,AHO,51
1,ALG,110
2,AND,5
3,ARG,7462
4,ARM,220
...,...,...
136,VEN,554
137,VIE,35
138,YUG,159
139,ZAM,13


In [18]:
#Tabellen joinen
df_merge2 = pd.merge(df_winner, df_loser , how="outer", left_on="winner_ioc", right_on="loser_ioc")   

#winner_ioc und loser_ioc zu einert Spalte vereinen, um alle Nationen in einer Spalte abzudecken
df_merge2['Nation'] = df_merge2['loser_ioc'].combine_first(df_merge2['winner_ioc'])

#Spalten umbenennen
df_merge2.rename(columns={"wins_count": "Siege", "loses_count": "Niederlagen"}, inplace=True)

#Spalten entfernen, die nicht relevant sind
df_merge2.drop(["winner_ioc", "loser_ioc"], axis=1, inplace=True)

#NaN-Werte der Spalte "Niederlagen" durch 0 ersetzen und Datentyp zu "int" ändern
fillna_0_astype_int64(df_merge2)

#Siegesquote bestimmen
siegesquote_funktion(df_merge2)

df_merge2

,Siege,Niederlagen,Nation,Siegesquote
0,29,51,AHO,0.362500
1,50,110,ALG,0.312500
2,3,5,AND,0.375000
3,8185,7462,ARG,0.523103
4,158,220,ARM,0.417989
...,...,...,...,...
137,0,4,PNG,0.000000
138,0,6,SUD,0.000000
139,0,1,SUR,0.000000
140,0,16,TRI,0.000000


<br>

## Prüfen, ob die geringe Siegesquote Deutschlands an der generellen Spielspärke der Gegnernation liegt

In [19]:
#Tabellen joinen, um generelle Sigesquote der Gegenernationen der Sigesquote von Deutschland gegen diese Nation gegenüberzustellen
df_merge3 = pd.merge(df_merge, df_merge2, how="outer", left_on="Gegnernation", right_on="Nation")

#Unnötige Spalten löschen
df_merge3.drop(["Deutschland","Gegnernation"], axis=1, inplace=True)

#Spalten umbennen
df_merge3.rename(columns={"Nation": "Gegnernation","Siege_y":"Siege_Gegnernation_allgemein", "Niederlagen_y":"Niederlagen_Gegnernation_allgemein","Siege_x": "Siege_Deutschland", "Niederlagen_x":"Niederlagen_Deutschland","Siegesquote_x": "Siegesquote_Deutschland_Gegnernation", "Siegesquote_y": "Siegesquote_Gegnernation_allgemein"}  ,inplace=True)

#Spaltenreihenfolge zur besseren Veranschaulichung ändern
df_merge3 = df_merge3[['Siege_Deutschland', 'Niederlagen_Deutschland', 'Siegesquote_Deutschland_Gegnernation', 'Gegnernation','Siege_Gegnernation_allgemein', 'Niederlagen_Gegnernation_allgemein', 'Siegesquote_Gegnernation_allgemein']]

#Spalte löschen bei der "Deutschland gegen Deuthschland spielt"
df_merge3.drop(df_merge3[df_merge3['Gegnernation'] == "GER"].index)

#Aufsteigend nach "Siegesquote_Deutschland_Gegnernation" und absteigend nach "Niederlagen_Deutschland" sortiert	
df_merge3 = df_merge3.sort_values(by=["Siegesquote_Deutschland_Gegnernation", "Niederlagen_Deutschland"], ascending=[True, False])

#NaN-Werte der Spalte "Niederlagen" durch 0 ersetzen und Datentyp zu "int" ändern
fillna_0_astype_int64(df_merge3, ["Niederlagen_Deutschland", "Siege_Deutschland"])


,Siege_Deutschland,Niederlagen_Deutschland,Siegesquote_Deutschland_Gegnernation,Gegnernation,Siege_Gegnernation_allgemein,Niederlagen_Gegnernation_allgemein,Siegesquote_Gegnernation_allgemein
92,0,1,0.000000,ESA,48,72,0.400000
20,12,36,0.250000,CYP,358,379,0.485753
87,11,31,0.261905,UZB,323,410,0.440655
49,7,15,0.318182,LTU,163,216,0.430079
59,3,6,0.333333,PAK,345,397,0.464960
...,...,...,...,...,...,...,...
137,0,0,NaN,LBA,0,8,0.000000
138,0,0,NaN,PNG,0,4,0.000000
139,0,0,NaN,SUR,0,1,0.000000
140,0,0,NaN,TRI,0,16,0.000000


In [20]:
#Filterung der Zeilen, bei denen Deutschland mehr als 5 Mal gegen diese Nation gespielt und die Gegenernation allgemein mehr 20 Spiele gespielt hat.
#Damit können z.B. Sigesquoten von 100% ausgeschlossen werden, bei denen nur sehr wenige Matches satttgefunden haben

df_filtered2 = df_merge3[((df_merge3.Siege_Deutschland + df_merge3.Niederlagen_Deutschland)>5) & ((df_merge3.Siege_Gegnernation_allgemein + df_merge3.Niederlagen_Gegnernation_allgemein)>20)]

#Top 5 Angstgegner von Deutschland
df_filtered2[0:5]

,Siege_Deutschland,Niederlagen_Deutschland,Siegesquote_Deutschland_Gegnernation,Gegnernation,Siege_Gegnernation_allgemein,Niederlagen_Gegnernation_allgemein,Siegesquote_Gegnernation_allgemein
20,12,36,0.250000,CYP,358,379,0.485753
87,11,31,0.261905,UZB,323,410,0.440655
49,7,15,0.318182,LTU,163,216,0.430079
59,3,6,0.333333,PAK,345,397,0.464960
51,25,43,0.367647,MAR,743,810,0.478429


### df_filtered2 als csv-Datei für Power BI abspeichern

In [21]:
df_filtered2.to_csv('PowerBI/Siegesquote_DEvsGEGNER.csv', index=False)

<br>

### Visualisierung der allgemeinen Siegesquote der "Angstgegner-Nationen"

In [22]:
#Allgemeine Siegesquote von Deutschland
df_merge2[df_merge2.Nation == "GER"]["Siegesquote"]

42    0.476651
Name: Siegesquote, dtype: float64

In [23]:
fig = px.bar(df_filtered2[0:5], x="Gegnernation", y=['Siegesquote_Deutschland_Gegnernation', 'Siegesquote_Gegnernation_allgemein'],
             barmode="group", title="Siegesquote_Deutschland_Gegnernation vs Siegesquote_Gegnernation_allgemein", labels={'value': 'Siegesquote', 'variable': 'Quelle'})
fig.show()

Auf Grundlage der Grafik kann man nochmals erkennen, dass jede allgemeine Siegesquote der "Angstgegner-Nationen" unter 50% ist und zudem ungefähr mit der von Deutschland übereinstimmen. Trotzdem sind alle Siegesquoten gegen die jewilige Nation relativ niedrig.
Daraus lässt sich auf den ersten Blick vermuten, dass die niedrige Siegesquote von Deutschland gegen die jeweilige "Angstgegner-Nation" nicht direkt auf die generelle Spielstärke dieser Nation und die Deutschlands zurückzuführen ist.

<br>

In [24]:
korrelation = df_filtered2["Siegesquote_Deutschland_Gegnernation"].corr(df_filtered2["Siegesquote_Gegnernation_allgemein"])
korrelation

-0.6742293150096238

Die Siegesquote_Deutschland_Gegnernation scheint die Siegesquote_Gegnernation_allgemein moderat bis stark negativ zu beeinflussen. Das bedeutet, dass die Siegesquote von Deutschland gegen die jeweilige Gegneration sinkt, wenn die allgemeine Siegesquote der Gegnernation steigt.
Also scheinen diese 5 Beispiele eher Ausreißer zu sein.

<br>

## Gibt es Lieblingsgegnernationen?

In [25]:
df_filtered2 = df_filtered2.sort_values(by=["Siegesquote_Deutschland_Gegnernation", "Siege_Deutschland"], ascending=[False, False])

#Top 5 "Lieblingsgegner-Nationen"
df_filtered2[0:5]

,Siege_Deutschland,Niederlagen_Deutschland,Siegesquote_Deutschland_Gegnernation,Gegnernation,Siege_Gegnernation_allgemein,Niederlagen_Gegnernation_allgemein,Siegesquote_Gegnernation_allgemein
36,7,0,1.000000,INA,164,273,0.375286
42,7,0,1.000000,JAM,86,227,0.274760
83,16,1,0.941176,UNK,60,360,0.142857
88,20,3,0.869565,VEN,335,554,0.376828
0,5,1,0.833333,ALG,50,110,0.312500


<br>

### Visualisierung der allgemeinen Siegesquote der "Lieblingsgegner-Nationen"

In [26]:
fig = px.bar(df_filtered2[0:5], x="Gegnernation", y=['Siegesquote_Deutschland_Gegnernation', 'Siegesquote_Gegnernation_allgemein'],
             barmode="group", title="Siegesquote_Deutschland_Gegnernation vs Siegesquote_Gegnernation_allgemein", labels={'value': 'Siegesquote', 'variable': 'Quelle'},)
fig.show()

Auf Grundlage der Grafik kann man erkennen, dass alle "Lieblinsggegner" von Deutschland auch generell eine sehr niedrige Siegesquote haben. Daraus kann man schließen, dass die hohe Siegesquote von Deutschland gegen die jeweilige "Lieblingsgegner-Nation" auf die geringe Spielstärke der Nation zurückzuführen ist.

<br>

### Data Frames speichern

In [27]:
%store df_matches
%store df_players
%store df_rankings

Stored 'df_matches' (DataFrame)
Stored 'df_players' (DataFrame)


Stored 'df_rankings' (DataFrame)
